In [32]:
import plotly.graph_objects as go
import geopandas as gpd
import dash
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

shapefile = 'Downloads/cb_2018_us_cd116_5m/cb_2018_us_cd116_5m.shp'

In [33]:
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()


In [54]:
flight_color_dict = {'AA': '#14ACDC', 'SW': 'red', 'UA': '#005daa', 'NK': '#ffcd41', 'DL': 'black', 'AS':'#7e8bfe'}
airport_alt_dict = {}
airport_lats     = []
airport_lons     = []
airport_codes    = []

for airport in fr_api.get_airports():
    if airport.country != 'United States':
        continue
    else:
        airport_alt_dict[airport.iata] = airport.altitude
airport_alt_dict['N/A'] = 0

In [55]:
flights = fr_api.get_flights(airline='AA')
fig = go.Figure()

fig.update_layout(
    title_text = 'Current Flights',
    showlegend = False,
    geo = go.layout.Geo(
        scope="usa",
        #projection_type = 'mercator',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        showsubunits=True,
        countrycolor = 'rgb(204, 204, 204)',
    ),
    height=700,
);

fig.update_geos(
    resolution=50,
    showcoastlines=True, coastlinecolor="black",
    showcountries=True, countrycolor="black",
    showland=True, landcolor= 'LightGreen',
    showocean=True, oceancolor="LightBlue",
);

airport_lats     = []
airport_lons     = []
airport_codes    = []

airports = {i.iata: i for i in fr_api.get_airports()}

for flight in flights: 
    if flight.altitude > 0: # in the air:
        if flight.destination_airport_iata != 'N/A':
            airport_lats.append(airports[flight.destination_airport_iata].latitude)
            airport_lons.append(airports[flight.destination_airport_iata].longitude)
            airport_codes.append(flight.destination_airport_iata)
        
        fig.add_trace(go.Scattergeo(
            lon = [flight.longitude],
            lat = [flight.latitude],
            hovertemplate='<b>Callsign: </b>' + flight.callsign 
                + '<br><b>Altitude: </b>'+ str(flight.altitude) + 'ft' 
                + '<br><b>Ground Speed: </b>' + flight.get_ground_speed() + '<extra></extra>'
                + '<br><b>Type: </b>' + flight.aircraft_code,
            mode = 'markers',
            marker = dict(
                size = 25,
                angle = flight.heading, 
                color = flight_color_dict[flight.airline_icao[0:2]], #'#a8a8a8',
                line=dict(width=2, color='black'),
                symbol = 'arrow',
                )
            ))
        
fig.add_trace(go.Scattergeo(
            #locationmode = 'USA-states',
            lon = airport_lons,
            lat = airport_lats,
            #hoverinfo = ['lat'],
            mode = 'markers',
            marker = dict(
                size = 5,
                color = 'black', #'#a8a8a8',
                line=dict(width=2, color='black'),
                )
            ))
fig.add_trace(go.Scattergeo(
            #locationmode = 'USA-states',
            lon = airport_lons,
            lat = [i + 0.5 for i in airport_lats],
            text= airport_codes, 
            #hoverinfo = ['lat'],
            mode = 'text',
#             marker = dict(
#                 size = 5,
#                 color = 'black', #'#a8a8a8',
#                 line=dict(width=2, color='black'),
#                 )
            ))
figLine = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces to the subplots
figLine.add_trace(
    go.Scatter(x=None, y=None, mode="lines+markers", name="Speed", line=dict(color="blue")),
    secondary_y=False,
);

figLine.add_trace(
    go.Scatter(x=None, y=None, mode="lines+markers", name="Altitude", line=dict(color="red")),
    secondary_y=True,
);

figHead = px.line_polar(r=None, theta=None)


In [56]:
# American, Southwest, United, Spirit
codes             = {'American': 'AA', 'Southwest': 'SW', 'United': 'UA', 'Spirit': 'NK', 'Delta': 'DL', 'Alaska':'AS'}
flight_color_dict = {'AA': '#14ACDC', 'SW': 'red', 'UA': '#005daa', 'NK': '#ffcd41', 'DL': 'black', 'AS':'#7e8bfe'}

In [57]:
from dash import Dash
from dash import dcc, ctx
from dash import html
from dash.exceptions import PreventUpdate

from dash import Output, Input, State

app = Dash()

In [58]:
app.layout = html.Div(children = [
    html.Div(children = [
            dcc.RadioItems(['American', 'Southwest', 'United', 'Spirit', 'Delta', 'Alaska'], 'Spirit', id = 'airlineSelect', inline=True),
            dcc.Input(id='destInput', type='text', value = None, placeholder="destination code", style = {'marginLeft':'25px'}), 
            dcc.Input(id='typeInput', type='text', value = None, placeholder="aircraft type", style = {'marginLeft':'25px'})], 
            style = {'display':'flex', 'flexDirection':'row'}),
    html.Div(children = dcc.Graph(figure = fig,     id = 'mapFig')), 
    html.Div([
        html.Div(children = dcc.Graph(figure = figLine, id = 'statPlot')), 
        html.Div(children = dcc.Graph(figure = figHead, id = 'headPlot'))
    ], style = {'display':'flex', 'flexDirection':'row'}), 

    dcc.Interval(interval=4000, id = 'reload'), 
    dcc.Store(id = 'altStore', data = {}),
    dcc.Store(id = 'speStore', data = {}), 
    dcc.Store(id = 'clickStore', data = None),
    dcc.Store(id = 'headStore', data = {}), 
    dcc.Store(id = 'airportAlts', data = airport_alt_dict), 
    dcc.Store(id = 'trackLats', data = []), 
    dcc.Store(id = 'trackLons', data = [])
])

In [59]:
@app.callback(Output(component_id = 'mapFig', component_property = 'figure'), 
              Output(component_id = 'altStore', component_property = 'data'), 
              Output(component_id = 'speStore', component_property='data'), 
              Output(component_id = 'headStore', component_property = 'data'),
              Output(component_id = 'trackLats', component_property = 'data'), 
              Output(component_id = 'trackLons', component_property = 'data'), 
             Input('reload', 'n_intervals'), 
             Input('airlineSelect', 'value'), 
             Input('mapFig', component_property = 'clickData'),
             Input('destInput', 'value'), 
             State(component_id = 'mapFig', component_property = 'figure'),
             State('altStore', component_property = 'data'), 
             State('speStore', component_property='data'), 
             State('headStore', component_property='data'), 
             State('airportAlts', component_property='data'), 
             State(component_id = 'clickStore',  component_property = 'data'), 
             State(component_id = 'trackLats', component_property = 'data'),
             State(component_id = 'trackLons', component_property = 'data')) 


def updateFigure(n_intervals, airline, click, destInput, figInit, altitudes, speeds, 
                 headings, airportAlts, callsign, trackLats, trackLons):  
    
    if ctx.triggered_id == 'mapFig':
        trackLats = []
        trackLons = []
        return figInit, altitudes, speeds, headings, trackLats, trackLons
    airlineCode = codes[airline]
    flights = fr_api.get_flights(airline=airlineCode)
    fig = go.Figure()

    

    fig.update_layout(
        title_text = 'Current Flights',
        showlegend = False,
        geo = go.layout.Geo(
            scope = 'usa',
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
        height=700,
    );

    fig.update_geos(
        resolution=50,
        showcoastlines=True, coastlinecolor="black",
        showcountries=True, countrycolor="black",
        showland=True, landcolor= 'LightGreen',
        showocean=True, oceancolor="LightBlue",
        showsubunits = True, subunitcolor = 'black'
    );
    
    airport_lats     = []
    airport_lons     = []
    airport_codes    = []

    for flight in flights: 
        airline_color = flight_color_dict[flight.airline_icao[0:2]]
        if flight.destination_airport_iata == 'N/A':
            continue
        if flight.destination_airport_iata not in list(airportAlts.keys()): # filtering for domestic flights
            continue
        if flight.origin_airport_iata not in list(airportAlts.keys()):
            continue
        if destInput: #(destInput is not None) and (destInput != ''): 
            if flight.destination_airport_iata != destInput:
                continue
            
        if flight.altitude > 0: # in the air:
            if flight.callsign in list(altitudes.keys()): 
                altitudes[flight.callsign].append(flight.altitude - airportAlts[flight.destination_airport_iata]) #- flight.destination_airport_altitude) #
                speeds[flight.callsign].append(flight.ground_speed)
                headings[flight.callsign].append(flight.heading)
                
            else:                
                altitudes[flight.callsign] = []
                speeds[flight.callsign]    = []
                headings[flight.callsign]  = []
                
            if flight.destination_airport_iata != 'N/A':
                airport_lats.append(airports[flight.destination_airport_iata].latitude)
                airport_lons.append(airports[flight.destination_airport_iata].longitude)
                airport_codes.append(flight.destination_airport_iata)
                
                airport_lats.append(airports[flight.origin_airport_iata].latitude)
                airport_lons.append(airports[flight.origin_airport_iata].longitude)
                airport_codes.append(flight.origin_airport_iata)
            if flight.callsign == callsign: 
                trackLats.append(flight.latitude)
                trackLons.append(flight.longitude)
            
            fig.add_trace(go.Scattergeo(
                locationmode = 'USA-states',
                lon = [flight.longitude],
                lat = [flight.latitude],
                customdata = [flight.callsign],
                #hoverinfo = ['lat'],
                hovertemplate='<b>Callsign: </b>' + flight.callsign 
                    + '<br><b>Altitude: </b>'+ str(flight.altitude) + 'ft' 
                    + '<br><b>Ground Speed: </b>' + flight.get_ground_speed() + '<extra></extra>'
                    + '<br><b>Type: </b>' + flight.aircraft_code,
                mode = 'markers',
                marker = dict(
                    size = 20,
                    angle = flight.heading, 
                    color = flight_color_dict[flight.airline_icao[0:2]], #'#a8a8a8',
                    symbol = 'arrow',
                    line=dict(width=2, color='black'),
                    )
                ))
    #airport_lats  = np.unique(np.array(airport_lats))
    #airport_lons  = np.unique(np.array(airport_lons))
    #airport_codes = np.unique(np.array(airport_codes))

    fig.add_trace(go.Scattergeo(
        lon = airport_lons,
        lat = airport_lats,
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'black', 
            line=dict(width=2, color='black'),
            )
        ))
    fig.add_trace(go.Scattergeo(
        lon = airport_lons,
        lat = [i + 0.25 for i in airport_lats],
        text= airport_codes, 

        mode = 'text',
        ))
    fig.layout.xaxis.fixedrange = True
    fig.layout.yaxis.fixedrange = True
    fig.layout.uirevision = 'dontchange'
    
    fig.add_trace(go.Scattergeo(
    lon = trackLons, 
    lat = trackLats, 
    mode = 'lines',
    marker = dict(
        color = airline_color, 
        line=dict(width=2, color='black'),
        )
    ))
                
    return fig, altitudes, speeds, headings, trackLats, trackLons

@app.callback(Output(component_id = 'statPlot',   component_property = 'figure'),  
              Output(component_id = 'headPlot',   component_property = 'figure'),
              Output(component_id = 'clickStore', component_property = 'data'),
              #Output(component_id = 'trackLats',  component_property = 'data'), 
              #Output(component_id = 'trackLons',  component_property = 'data'), 
              Input(component_id = 'mapFig',      component_property = 'clickData'), 
              Input(component_id = 'reload',      component_property = 'n_intervals'), 
              State(component_id = 'altStore',    component_property = 'data'), 
              State(component_id = 'speStore',    component_property = 'data'), 
              State(component_id = 'headStore',   component_property = 'data'),
              State(component_id = 'clickStore',  component_property = 'data'), 
              
              prevent_initial_call = True) 
              
def updateLine(click, n, altitudes, speeds, headings, prevClick): 
    
    if (ctx.triggered_id == 'reload') and (prevClick is not None): 
        callsign = prevClick
    elif (ctx.triggered_id == 'reload') and (prevClick is None):
        raise PreventUpdate 
    else:
        #print(click)
        try:
            callsign = click['points'][0]['customdata']
        except:
            raise PreventUpdate

    if n == None:
        raise PreventUpdate
    
    x = range(n)
    
    from plotly.subplots import make_subplots
    import numpy as np
    
    figL = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add traces to the subplots
    figL.add_trace(
        go.Scatter(x=np.array(x), y=altitudes[callsign], mode="lines+markers", name="Altitude", line=dict(color="blue")),
        secondary_y=False,
    )

    figL.add_trace(
        go.Scatter(x=np.array(x), y=speeds[callsign], mode="lines+markers", name="Speed", line=dict(color="red")),
        secondary_y=True,
    )
    figL.update_layout(title = callsign)
    figHead = px.line_polar(r=speeds[callsign], theta=headings[callsign])
    #print(figL)
    return figL, figHead, callsign

In [60]:
if __name__ == '__main__':
    app.run(mode='external')